In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
df_green = spark.read.parquet("data/pq/green/*/*")

26/01/18 13:03:41 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/pq/green/*/*.
java.io.FileNotFoundException: File data/pq/green/*/* does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:980)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1301)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:970)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.sinks.FileStreamSink$.hasMetadata(FileStreamSink.scala:58)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:384)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.ResolveData

In [4]:
df_green.registerTempTable('green')

/opt/homebrew/Cellar/apache-spark/4.1.0/libexec/python/lib/pyspark.zip/pyspark/sql/classic/dataframe.py:178: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.


In [13]:
df_green_revenue = spark.sql("""
SELECT 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    SUM(total_amount) AS amount,
    COUNT(1) as number_records

FROM 
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
    ORDER BY 2,1
""")

In [14]:
df_green_revenue.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
|   7|2020-01-01 00:00:00| 769.7299999999997|            45|
|  17|2020-01-01 00:00:00|195.03000000000006|             9|
|  18|2020-01-01 00:00:00|               7.8|             1|
|  22|2020-01-01 00:00:00|              15.8|             1|
|  24|2020-01-01 00:00:00|              87.6|             3|
|  25|2020-01-01 00:00:00| 531.0000000000001|            26|
|  29|2020-01-01 00:00:00|              61.3|             1|
|  32|2020-01-01 00:00:00| 68.94999999999999|             2|
|  33|2020-01-01 00:00:00|317.27000000000004|            11|
|  35|2020-01-01 00:00:00|            129.96|             5|
|  36|2020-01-01 00:00:00|            295.34|            11|
|  37|2020-01-01 00:00:00|            175.67|             6|
|  38|2020-01-01 00:00:00| 98.78999999999999|             2|
|  40|2020-01-01 00:00:0

In [19]:
df_green_revenue.write.parquet('data/report/revenue/green')

26/01/18 13:46:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [20]:
print(spark.conf.get("spark.sql.adaptive.enabled"))
print(spark.conf.get("spark.sql.adaptive.coalescePartitions.enabled"))

true
true


In [21]:
df_yellow = spark.read.parquet("data/pq/yellow/*/*")

26/01/18 13:47:16 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: data/pq/yellow/*/*.
java.io.FileNotFoundException: File data/pq/yellow/*/* does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:980)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1301)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:970)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.sinks.FileStreamSink$.hasMetadata(FileStreamSink.scala:58)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:384)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.ResolveDa

In [22]:
df_yellow.registerTempTable('yellow')

In [23]:
df_yellow_revenue = spark.sql("""
SELECT 
    PULocationID AS zone,
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    SUM(total_amount) AS amount,
    COUNT(1) as number_records

FROM 
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1,2
    ORDER BY 2,1
""")

In [24]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow')

26/01/18 13:48:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                